In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# 리뷰 수집 함수
def collect_reviews(url, max_reviews=100):
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    # options.add_argument("--headless=new")  # 디버깅 중일 경우 주석처리
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)

    try:
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))
        driver.switch_to.frame("entryIframe")
    except:
        print(f"❌ iframe 전환 실패: {url}")
        driver.quit()
        return []

    time.sleep(3)

    for _ in range(20):
        try:
            more_btns = driver.find_elements(By.CSS_SELECTOR, "a.fvwqf")
            if more_btns:
                driver.execute_script("arguments[0].click();", more_btns[0])
                time.sleep(1)
            driver.execute_script("window.scrollBy(0, 2500);")
            time.sleep(2)
        except:
            break

    reviews = []
    review_tags = driver.find_elements(By.CSS_SELECTOR, "div.pui__vn15t2 > a[data-pui-click-code='rvshowmore']")
    for r in review_tags:
        reviews.append(r.text.strip())

    driver.quit()
    return reviews[:max_reviews]

# 리뷰 URL 목록 불러오기
df = pd.read_csv("건대 입구 카페 URL 200개.csv")
df["review_url"] = df["URL"].apply(lambda x: f"https://map.naver.com/p/search/건대입구 카페/place/{x.split('place/')[1].split('?')[0]}?c=15.00,0,0,0,dh&placePath=/review")
urls = df["review_url"].dropna().tolist()

# 모든 리뷰 수집
result = []
for url in urls:
    reviews = collect_reviews(url)
    for review in reviews:
        result.append({"url": url, "review": review})
    print(f"✅ 수집 완료: {url} → {len(reviews)}개")

# 저장
review_df = pd.DataFrame(result)
review_df.to_csv("건대입구_카페_리뷰수집_V2.csv", index=False, encoding="utf-8-sig")
print("🎉 전체 리뷰 수집 완료 및 저장!")
